In [1]:
#Import libraries
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string

In [2]:
#Load the data
df = pd.read_csv('emails.csv') #read the CSV file
df.head(5)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [3]:
#Print the shape (Get the number of rows and cols)
df.shape

(5728, 2)

In [4]:
#Get the column names
df.columns

Index(['text', 'spam'], dtype='object')

In [5]:
#Checking for duplicates and removing them
df.drop_duplicates(inplace = True)

In [6]:
#Show the new shape (number of rows & columns)
df.shape


(5695, 2)

In [7]:
#Show the number of missing (NAN, NaN, na) data for each column
df.isnull().sum()

text    0
spam    0
dtype: int64

In [8]:
#Need to download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
#Tokenization (a list of tokens), will be used as the analyzer
#1.Punctuations are [!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]
#2.Stop words in natural language processing, are useless words (data).
def process_text(text):
    '''
    What will be covered:
    1. Remove punctuation
    2. Remove stopwords
    3. Return list of clean text words
    '''
    
    #1
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    #2
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    #3
    return clean_words

In [10]:
#Show the Tokenization (a list of tokens )
df['text'].head().apply(process_text)

0    [Subject, naturally, irresistible, corporate, ...
1    [Subject, stock, trading, gunslinger, fanny, m...
2    [Subject, unbelievable, new, homes, made, easy...
3    [Subject, 4, color, printing, special, request...
4    [Subject, money, get, software, cds, software,...
Name: text, dtype: object

In [11]:
'''
EXAMPLE OF THE PROCESS TO PREPARE THE DATA FOR TRAINING ON THE CLASSIFIER, 
THIS CELL/BLOCK ISNT NECESSARY TO RUN THE PROGRAM
'''
#Print the text (aka the email message)
message4 = 'hello world hello hello world play' #df['text'][3]
message5 = 'test test test test one hello'
print(message4)
print()

#Convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer
bow4 =  CountVectorizer(analyzer=process_text).fit_transform([[message4], [message5]])
print(bow4)
print()

#Transform a count matrix to a normalized tf or tf-idf representation
#Show the weight of TF-IDF
#Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency. 
#from sklearn.feature_extraction.text import TfidfTransformer
#tfidf4 = TfidfTransformer().fit_transform(bow4)
#print(tfidf4)
#print()

#Print the shape (number of rows & columns) of bow4 
#print(bow4.shape)
#print()

#Show bow4 row at index 1
#bow4[1]

hello world hello hello world play

  (0, 2)	1
  (0, 4)	2
  (0, 0)	3
  (1, 1)	1
  (1, 3)	4
  (1, 0)	1



In [12]:
# Convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer

## Converts strings to integer counts
#vectorizer = CountVectorizer(analyzer=process_text)

## Learn a vocabulary dictionary of all tokens in the raw documents.
#bow_transformer = vectorizer.fit(df['text'])    

## Transform documents to document-term matrix.
#messages_bow = bow_transformer.transform(df['text'])

#Convert string to integer counts, learn the vocabulary dictionary and return term-document matrix
#---------------------------- IMPORTANT BELOW -------------------

##### messages_bow = CountVectorizer(analyzer=process_text).fit_transform(df['text'])

In [18]:
type(messages_bow)
from scipy import sparse

sparse.save_npz("yourmatrix.npz", messages_bow)
your_matrix_back = sparse.load_npz("yourmatrix.npz")

In [22]:
messages_bow=your_matrix_back

In [23]:
# Transform a count matrix to a normalized tf or tf-idf representation
# Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency. 
#from sklearn.feature_extraction.text import TfidfTransformer

## Learn the idf vector (global term weights)
#tfidf_transformer=TfidfTransformer().fit(messages_bow)

## Transform a count matrix to a tf or tf-idf representation
#messages_tfidf = tfidf_transformer.transform(messages_bow)

#Learn the idf vector to fit to data, then transform it.
#messages_tfidf = TfidfTransformer().fit_transform(messages_bow)

In [24]:
#Split the data into 80% training (X_train & y_train) and 20% testing (X_test & y_test) data sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(messages_bow, df['spam'], test_size = 0.20, random_state = 0)

In [25]:
messages_bow.shape


(5695, 37229)

In [26]:
#Create and train the Naive Bayes classifier
#The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification)
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
#Print the predictions
print(classifier.predict(X_train))

#Print the actual values
print(y_train.values)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [28]:
#Evaluate the model on the training data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train ,pred ))
print('Confusion Matrix: \n',confusion_matrix(y_train,pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       0.99      1.00      0.99      1099

   micro avg       1.00      1.00      1.00      4556
   macro avg       0.99      1.00      1.00      4556
weighted avg       1.00      1.00      1.00      4556

Confusion Matrix: 
 [[3445   12]
 [   1 1098]]

Accuracy:  0.9971466198419666


In [29]:
#Print the predictions
print('Predicted value: ',classifier.predict(X_test))

#Print Actual Label
print('Actual value: ',y_test.values)

Predicted value:  [1 0 0 ... 0 0 0]
Actual value:  [1 0 0 ... 0 0 0]


In [30]:
#Evaluate the model on the test data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_test)
print(classification_report(y_test ,pred ))

print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       0.97      1.00      0.98       269

   micro avg       0.99      0.99      0.99      1139
   macro avg       0.98      0.99      0.99      1139
weighted avg       0.99      0.99      0.99      1139

Confusion Matrix: 
 [[862   8]
 [  1 268]]

Accuracy:  0.9920983318700615
